In [1]:
!pip install roboflow

  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.3 kB)
  Using cached pillow-11.3.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.0 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached PyYAML-6.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached contourpy-1.3.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached charset_normalizer-3.4.3-cp313-cp313-macosx_10_13_universal2.whl.metadata (36 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 24.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.4/642.4 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [42]:
ROBOFLOW_API_KEY = "bqQx7aG8EwPnLZVzCCws"

PosixPath('/home/jamradise/MegaDendrologAI/ML/Detection/data/tree')

In [19]:
from roboflow import Roboflow
from pathlib import Path
PATH_TO_SAVE_DATASET = Path("/home/jamradise/MegaDendrologAI/ML/Detection/Data")
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("tree-detection-v4x2j").project("tree-quality-3")
version = project.version(7)
dataset = version.download("yolov11", location=str(PATH_TO_SAVE_DATASET / "trees-1"))

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /home/jamradise/MegaDendrologAI/ML/Detection/Data/trees-1 in yolov11:: 100%|██████████████████████████████████████████████████████| 1748/1748 [00:00<00:00, 8328.85it/s]


In [24]:
project = rf.workspace("govind-rai-aiwsm").project("shrubs")
version = project.version(1)
dataset = version.download("yolov11", location=str(PATH_TO_SAVE_DATASET / "shrubs-1"))

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /home/jamradise/MegaDendrologAI/ML/Detection/Data/shrubs-1 in yolov11:: 100%|█████████████████████████████████████████████████████| 1309/1309 [00:00<00:00, 6688.90it/s]


In [27]:
project = rf.workspace("mtlworkspace").project("bush-detection")
version = project.version(7)
dataset = version.download("yolov11", location=str(PATH_TO_SAVE_DATASET / "shrubs-2"))

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /home/jamradise/MegaDendrologAI/ML/Detection/Data/shrubs-2 in yolov11:: 100%|███████████████████████████████████████████████████████| 494/494 [00:00<00:00, 9180.31it/s]


In [26]:
project = rf.workspace("mohamed-snoqx").project("tree-6szhy")
version = project.version(1)
dataset = version.download("yolov11", location=str(PATH_TO_SAVE_DATASET / "trees-2"))

loading Roboflow workspace...
loading Roboflow project...


# Concat all datasets

In [31]:
trees_paths = [str(PATH_TO_SAVE_DATASET / "trees-1"), str(PATH_TO_SAVE_DATASET / "trees-2")]
shrubs = [str(PATH_TO_SAVE_DATASET / "shrubs-1"), str(PATH_TO_SAVE_DATASET / "shrubs-2")]

In [33]:
trees_paths, shrubs

(['/home/jamradise/MegaDendrologAI/ML/Detection/Data/trees-1',
  '/home/jamradise/MegaDendrologAI/ML/Detection/Data/trees-2'],
 ['/home/jamradise/MegaDendrologAI/ML/Detection/Data/shrubs-1',
  '/home/jamradise/MegaDendrologAI/ML/Detection/Data/shrubs-2'])

In [43]:
import os
from pathlib import Path
import shutil
import yaml

In [45]:
PATH_TO_SAVE_DATASET = Path("/home/jamradise/MegaDendrologAI/ML/Detection/Data")
new_dataset_path = PATH_TO_SAVE_DATASET / "combined_dataset"
new_dataset_path.mkdir(parents=True, exist_ok=True)

In [46]:
for split in ["train", "valid", "test"]:
    (new_dataset_path / split / "images").mkdir(parents=True, exist_ok=True)
    (new_dataset_path / split / "labels").mkdir(parents=True, exist_ok=True)

In [47]:
datasets = [
    {"path": PATH_TO_SAVE_DATASET / "trees-1", "type": "tree"},
    {"path": PATH_TO_SAVE_DATASET / "shrubs-1", "type": "shrub"},
    {"path": PATH_TO_SAVE_DATASET / "shrubs-2", "type": "shrub"},
    {"path": PATH_TO_SAVE_DATASET / "trees-2", "type": "tree"}
]

In [48]:
for ds in datasets:
    ds_path = ds["path"]
    ds_type = ds["type"]
    new_class_id = 0 if ds_type == "tree" else 1  # tree=0, shrub=1

    for split in ["train", "valid", "test"]:
        src_images = ds_path / split / "images"
        src_labels = ds_path / split / "labels"
        
        # Пропускаем, если папки не существуют
        if not src_images.exists() or not src_labels.exists():
            continue

        # Обрабатываем каждый файл в split
        for img_file in src_images.iterdir():
            base_name = img_file.stem
            ext = img_file.suffix
            new_img_name = f"{ds_path.name}_{base_name}{ext}"
            dest_img = new_dataset_path / split / "images" / new_img_name
            
            # Копируем изображение
            shutil.copy(img_file, dest_img)
            
            # Обрабатываем аннотацию
            label_file = src_labels / f"{base_name}.txt"
            if label_file.exists():
                with open(label_file, 'r') as f:
                    lines = f.readlines()
                
                new_lines = []
                for line in lines:
                    parts = line.strip().split()
                    if not parts:  # пропускаем пустые строки
                        continue
                    # Заменяем class_id на новый
                    parts[0] = str(new_class_id)
                    new_lines.append(" ".join(parts))
                
                new_label_name = f"{ds_path.name}_{base_name}.txt"
                dest_label = new_dataset_path / split / "labels" / new_label_name
                with open(dest_label, 'w') as f:
                    f.write("\n".join(new_lines))

In [49]:
data_yaml = {
    "path": str(new_dataset_path),
    "train": "train/images",
    "valid": "valid/images",
    "test": "test/images",
    "names": {
        0: "tree",
        1: "shrub"
    }
}

In [50]:
with open(new_dataset_path / "data.yaml", 'w') as f:
    yaml.dump(data_yaml, f, sort_keys=False)